In [2]:
import plotly.graph_objects as go
import numpy as np
import os

# モデルのパスや埋め込み範囲などのパラメータの設定
model_path = '/home/lab/eight/KGE-HAKE/models/HAKE1_wn18rr_0'
embedding_range = 0.01
head_id = 22311 #22311	mammal.n.01 16430	food.n.01
tail_id = 27	#27	dog.n.01 40630	pizza.n.01
num_bins = 250  # ヒストグラムのビンの数
model = 'HAKE1'
# エンティティ名の辞書読み込み
entities_dict_file = '/home/lab/eight/KGE-HAKE/data/wn18rr_text/entities.dict'
entities_dict = {}
with open(entities_dict_file, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        entities_dict[int(parts[0])] = parts[1]

In [12]:
# エンティティの埋め込みをロード
entity_embedding = np.load(os.path.join(model_path, 'entity_embedding.npy'))

# HeadとTailの座標計算
head = entity_embedding[head_id]
tail = entity_embedding[tail_id]

phase_head, mod_head = np.split(head, 2)
phase_tail, mod_tail = np.split(tail, 2)

mod_head = np.log(np.abs(mod_head)) * np.sign(mod_head)
mod_tail = np.log(np.abs(mod_tail)) * np.sign(mod_tail)

phase_head = phase_head / embedding_range * np.pi
phase_tail = phase_tail / embedding_range * np.pi

x_head, y_head = mod_head * np.cos(phase_head), mod_head * np.sin(phase_head)
x_tail, y_tail = mod_tail * np.cos(phase_tail), mod_tail * np.sin(phase_tail)

fig = go.Figure()

# Head Entityのプロット
fig.add_trace(go.Scatter(x=x_head, y=y_head, mode='markers', name=entities_dict[head_id], text=[f'Head ID: {head_id}'] * len(x_head), opacity=0.8))
# Tail Entityのプロット
fig.add_trace(go.Scatter(x=x_tail, y=y_tail, mode='markers', name=entities_dict[tail_id], text=[f'Tail ID: {tail_id}'] * len(x_tail), opacity=0.8))
# HAKEテキストの追加
fig.add_annotation(
    text=model,  # 表示するテキスト
    xref="paper",  # x座標の参照元 (paperは図の左端、dataはデータ座標)
    yref="paper",  # y座標の参照元 (paperは図の下端、dataはデータ座標)
    x=0.5,  # x座標 (0.5は図の中央)
    y=1.1,  # y座標 (1.05は図の上端から5%上)
    showarrow=False,  # 矢印を非表示にする
    font=dict(size=20),  # テキストのフォントサイズ
)
# レイアウトの設定
fig.update_layout(
    title="",
    yaxis_title=" ",
    showlegend=True,
    width=600,  # 幅
    height=550,  # 高さ
    xaxis=dict(scaleanchor="y", scaleratio=1),  # x軸のアスペクト比を1:1に設定
    yaxis=dict(scaleanchor="x", scaleratio=1),  # y軸のアスペクト比を1:1に設定
)

# プロットの表示
fig.show()


In [5]:

# headとtailの各座標を計算
phase_head, mod_head = np.split(entity_embedding[head_id], 2)
phase_tail, mod_tail = np.split(entity_embedding[tail_id], 2)

# ログ変換と極座標への変換
mod_head = np.log(np.abs(mod_head)) * np.sign(mod_head)
mod_tail = np.log(np.abs(mod_tail)) * np.sign(mod_tail)

phase_head = phase_head / embedding_range * np.pi
phase_tail = phase_tail / embedding_range * np.pi

# 極座標から直交座標への変換
x_head, y_head = mod_head * np.cos(phase_head), mod_head * np.sin(phase_head)
x_tail, y_tail = mod_tail * np.cos(phase_tail), mod_tail * np.sin(phase_tail)

# 距離の計算
distance_head = np.sqrt(x_head**2 + y_head**2)
distance_tail = np.sqrt(x_tail**2 + y_tail**2)

# 指定された区間で均等に分割
bin_edges = np.linspace(0, np.max([np.max(mod_head), np.max(mod_tail)]), num_bins + 1)

# ヒストグラムの作成
hist_head = np.histogram(mod_head, bins=bin_edges)
hist_tail = np.histogram(mod_tail, bins=bin_edges)

# Plotlyを使用してヒストグラムをプロット
fig = go.Figure()

# Head Entityのヒストグラム
fig.add_trace(go.Bar(x=hist_head[1][:-1], y=hist_head[0], name="head "+entities_dict[head_id], opacity=1.0, width=0.05))

# Tail Entityのヒストグラム
fig.add_trace(go.Bar(x=hist_tail[1][:-1], y=hist_tail[0], name="tail "+entities_dict[tail_id], opacity=1.0, width=0.05))

# レイアウトの設定
fig.update_layout(
    title='HAKE',
    xaxis=dict(title='modulus 絶対値の大きさ'),
    yaxis=dict(title='要素 [hm/tm]i のカウント'),
    barmode='overlay',  # HeadとTailのヒストグラムをオーバーレイ
)

# プロットの表示
fig.show()


In [14]:
# 平均距離の計算
mean_distance_head = np.mean(np.sqrt(x_head**2 + y_head**2))
mean_distance_tail = np.mean(np.sqrt(x_tail**2 + y_tail**2))

# 結果の出力
print(f"平均距離（Head）: {mean_distance_head}")
print(f"平均距離（Tail）: {mean_distance_tail}")

平均距離（Head）: 11.932021141052246
平均距離（Tail）: 4.860427379608154


In [15]:
# カウントの合計数を計算
head_count = np.sum(hist_head[0])
tail_count = np.sum(hist_tail[0])

# 結果の出力
print(f"表示Entity（Head）: {head_count}")
print(f"表示Entity（Tail）: {tail_count}")

表示Entity（Head）: 500
表示Entity（Tail）: 500
